<a href="https://colab.research.google.com/github/MIAShield/MIAShield/blob/main/Cifar10_and_Cifar100_Related_Work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#install all required library
!sudo pip install Pillow
!sudo pip install imagehash

In [ ]:
# install the most recent verson of tensorflow-privacy
#!pip install -U git+https://github.com/tensorflow/privacy
!pip install tensorflow-privacy

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import glob
import itertools
import collections

from PIL import Image
import cv2
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import numpy as np
import torch
import imagehash
import hashlib


#importing libraries
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
import sys
from matplotlib import pyplot
from keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from tensorflow.keras.optimizers import SGD
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# general imports
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from scipy import special

# tensorflow imports
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, MaxPooling2D,Dropout, BatchNormalization, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical


In [ ]:
# tensorflow-privacy libraries
from tensorflow_privacy.privacy.membership_inference_attack import membership_inference_attack as mia
from tensorflow_privacy.privacy.membership_inference_attack.data_structures import AttackInputData
from tensorflow_privacy.privacy.membership_inference_attack.data_structures import SlicingSpec
from tensorflow_privacy.privacy.membership_inference_attack.data_structures import AttackType

/usr/local/lib/python3.7/dist-packages/tensorflow_privacy/privacy/membership_inference_attack/__init__.py:19: UserWarning: 
Membership inference attack sources were moved. Please replace
import tensorflow_privacy.privacy.membership_inference_attack

with
import tensorflow_privacy.privacy.privacy_tests.membership_inference_attack
  "\nMembership inference attack sources were moved. Please replace"


In [ ]:
dataset='Cifar100'    #dataset 'Cifar10' or 'Cifar100'

In [ ]:
if dataset=='Cifar10':
  #define all the variables
#cifar10
  shape = (32, 32, 3)
  num_class=10
  learning_rate=.01
  epochs=60
elif dataset=='Cifar100':
  #cifar100
  shape = (32, 32, 3)
  num_class=100
  learning_rate=.01
  epochs=130


In [ ]:
batch_size=128

In [ ]:
def load_cifar10():
  """Loads cifar10-Dataset and preprocesses to combine training and test data."""
  
  # load the existing CIFAR10 dataset that comes in form of traing + test data and labels
  train, test = tf.keras.datasets.cifar10.load_data()
  train_data, train_labels = train
  test_data, test_labels = test

  # scale the images from color values 0-255 to numbers from 0-1 to help the training process
  train_data1 = np.array(train_data, dtype=np.float32) / 255
  test_data1 = np.array(test_data, dtype=np.float32) / 255

  # cifar10 labels come one-hot encoded, there
  train_labels = train_labels.flatten()
  test_labels = test_labels.flatten()

  return train_data1, train_labels, test_data1, test_labels, train_data, train_labels, test_data, test_labels 

In [ ]:
def load_cifar100():
  """Loads cifar100-Dataset and preprocesses to combine training and test data."""
  
  # load the existing CIFAR10 dataset that comes in form of traing + test data and labels
  train, test = tf.keras.datasets.cifar100.load_data()
  train_data, train_labels = train
  test_data, test_labels = test

  # scale the images from color values 0-255 to numbers from 0-1 to help the training process
  train_data1 = np.array(train_data, dtype=np.float32) / 255
  test_data1 = np.array(test_data, dtype=np.float32) / 255

  # cifar10 labels come one-hot encoded, there
  train_labels = train_labels.flatten()
  test_labels = test_labels.flatten()

  return train_data1, train_labels, test_data1, test_labels, train_data, train_labels, test_data, test_labels 

In [ ]:
if dataset=='Cifar10':
       train_data, train_labels, test_data, test_labels,trainX1,trainY1, testX1,testY1 = load_cifar10()  
elif dataset=='Cifar100':
       train_data, train_labels, test_data, test_labels,trainX1,trainY1, testX1,testY1 = load_cifar100() 


169017344/169001437 [==============================] - 3s 0us/step


In [ ]:
train_y=to_categorical(train_labels)
test_y=to_categorical(test_labels)

In [ ]:
def AlexnetModel(input_shape,num_classes):
    model = Sequential()
# model.add(Conv2D(96, (11,11), strides=(4,4), activation='relu', padding='same', input_shape=(img_height, img_width, channel,)))
# for original Alexnet
    model.add(Conv2D(48, (3,3), strides=(2,2), activation='relu', padding='same', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
# Local Response normalization for Original Alexnet
    model.add(BatchNormalization())

    model.add(Conv2D(96, (3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))
# Local Response normalization for Original Alexnet
    model.add(BatchNormalization())

    model.add(Conv2D(192, (3,3), activation='relu', padding='same'))
    model.add(Conv2D(192, (3,3), activation='relu', padding='same'))
    model.add(Conv2D(256, (3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))
# Local Response normalization for Original Alexnet
    model.add(BatchNormalization())

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    return model

In [ ]:
# make the neural network model with the function specified above.
# one model is supposed to train for 10, one for 50 epochs
model_np = AlexnetModel(shape,num_class)
model_np.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 16, 16, 48)        1344      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 8, 8, 48)         0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 8, 8, 48)         192       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 8, 8, 96)          41568     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 3, 3, 96)         0         
 2D)                                                             
                                                        

In [ ]:
#Run the model
# specify parameters
#learning_rate=.005
optimizer = tf.keras.optimizers.SGD(learning_rate)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

# compile the model
model_np.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [ ]:
# train the model
history = model_np.fit(train_data, train_y,
                       validation_data=(test_data, test_y),
                       batch_size=128, 
                       epochs=epochs)

In [ ]:
# since we have not specified an activation function on the last layer
# calling the predict function returns the logits
print('Predict on train...')
prob_train = model_np.predict(train_data[0:5000])
print('Predict on test...')
prob_test = model_np.predict(test_data[5000:10000])

Predict on train...
Predict on test...


In [ ]:
train=train_data[0:5000]
test=test_data[5000:10000]

In [ ]:
stop!!

DP-SGD

In [ ]:
# general imports
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from scipy import special

# tensorflow imports
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, MaxPooling2D,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy
from tensorflow_privacy.privacy.optimizers.dp_optimizer import DPGradientDescentGaussianOptimizer,DPAdamGaussianOptimizer
import tensorflow_privacy
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy

In [ ]:
l2_norm_clip = 1.5
noise_multiplier = .248
num_microbatches = 16
learning_rate = 0.01
batch_size=128

if batch_size % num_microbatches != 0:
  raise ValueError('Batch size should be an integer multiple of the number of microbatches')

optimizer = tensorflow_privacy.DPKerasSGDOptimizer(
    l2_norm_clip=l2_norm_clip,
    noise_multiplier=noise_multiplier,
    num_microbatches=num_microbatches,
    learning_rate=learning_rate)

loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True, reduction=tf.losses.Reduction.NONE)
#loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
modelsgd=AlexnetModel(shape,num_class)
modelsgd.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [ ]:
compute_dp_sgd_privacy.compute_dp_sgd_privacy(n=train_data.shape[0],
                                              batch_size=batch_size,
                                              noise_multiplier=.248,
                                              epochs=epochs,
                                              delta=1e-5)

In [ ]:
history=modelsgd.fit(train_data, train_y,
          epochs=epochs,
          validation_data=(test_data[0:1000], test_y[0:1000]),
          batch_size=128)

In [ ]:
# since we have not specified an activation function on the last layer
# calling the predict function returns the logits
print('Predict on train...')
prob_trainsgd = modelsgd.predict(train_data[0:5000])
print('Predict on test...')
prob_testsgd = modelsgd.predict(test_data[5000:10000])
print('Compute losses...')
cce = tf.keras.backend.categorical_crossentropy
constant = tf.keras.backend.constant
yt=train_labels[0:5000]
ys=test_labels[5000:10000]
y_train_onehot = to_categorical(yt)
y_test_onehot = to_categorical(ys)

#loss_train = cce(constant(y_train_onehot), constant(prob_train), from_logits=False).numpy()
#loss_test = cce(constant(y_test_onehot), constant(prob_test), from_logits=False).numpy()

In [ ]:
stop!!

PATE

In [ ]:
#from imblearn.over_sampling import RandomOverSampler

#ros = RandomOverSampler()

#X_ros, y_ros = ros.fit_resample(x_train_split0, trainy_split0)
# split x_train to 5 disjoint datasets
#store each dataset variable name is the list Xtrain
M=25
from sklearn.model_selection import StratifiedKFold
X=train_data
Y=train_labels
for i in range(0,M):
  K=M-i
  if(K>=2):
    train=X
    test=Y
    skf = StratifiedKFold(n_splits=K)
    skf.get_n_splits(X, Y)
    #print(skf)
    for train_index, test_index in skf.split(X, Y):
       #print(X.shape)
       X,globals()['x_train_pate%s' % i]=train[train_index], train[test_index]
       Y,globals()['patey_split%s' % i]=test[train_index], test[test_index]
       y_hot=globals()['patey_split%s' % i]
       y_hot=to_categorical(y_hot)
       globals()['y_train_pate%s' % i]=y_hot
       #X=train
       #Y=test
  elif K==1:
    globals()['x_train_pate%s' % i]=X
    globals()['patey_split%s' % i]=Y
    y_hot=to_categorical(Y)
    globals()['y_train_pate%s' % i]=y_hot

In [ ]:
(unique, counts) = np.unique(patey_split0, return_counts=True)
frequencies = np.asarray((unique, counts)).T
frequencies

In [ ]:
model_index =0
for iter in range(M):
  modelpate=AlexnetModel(shape,num_class)
  #modelpate.build(input_shape = (None,32,32,3))#AlexnetModel(shape,num_class)
  optimizer = tf.keras.optimizers.SGD(learning_rate)
  loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
  modelpate.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
  modelpate.fit(globals()['x_train_pate%s' % iter], globals()['y_train_pate%s' % iter], validation_data=(test_data, test_y),
                       batch_size=128, 
                       epochs=epochs)
  globals()['model_pate%s' % iter]=modelpate

In [ ]:
#calculate accuracy/utility test acc
prb=model_pate29.predict(test_data)
test_labeld_result=np.argmax(prb,axis=1)
accuracy_score(test_labels,test_labeld_result)
#prob_testd

In [ ]:
def confidence(my_list):
  import numpy as np
  my_array=np.array(my_list)
  p=np.zeros(my_array.shape)
  b=my_array.max(-1)
  condition = my_array == b[..., np.newaxis]
  c = np.where(condition, 1, 0)
  final=np.multiply(c, my_array)
  #my_sum=np.sum(final,axis=0)
  labels=np.argmax(final, axis=1)
  return final

In [ ]:
#calculate parameters from our combined
#from joblib import Parallel, delayed
import time
def calc_pate(data1,ep):
  n=len(data1)
  #logits=np.zeros((n,10))
  prob=np.zeros((n,num_class))
  labels=np.zeros(n)
  #index_resulted=np.zeros(n)
  #ts=.02
  kk=0
  sum2=np.zeros((n,num_class))
    #index=model_index(d2,5)
    #index_resulted[p1]=index
    #print(index)
  for p2 in range(M):
        modelh=globals()['model_pate%s' % p2]
        prob_d = modelh.predict(data1)
        #prob_d = special.softmax(logits_d)
        kk=kk+1
        #sum1=logits_d+sum1
        prob_vote=vote(prob_d)
        sum2=prob_vote+sum2
  sum_noise=sum2+np.random.laplace(loc=0.0, scale=1/ep)
  prob=sum_noise/M
  prob=vote(prob)
    #labels[p1,:]=np.argmax(logits[p1,:])
  return prob


In [ ]:
ep=[0.01,1,100]

In [ ]:
def vote(my_list):
  import numpy as np
  my_array=np.array(my_list)
  p=np.zeros(my_array.shape)
  b=my_array.max(-1)
  condition = my_array == b[..., np.newaxis]
  c = np.where(condition, 1, 0)
  final=np.multiply(c, my_array)
  #my_sum=np.sum(final,axis=0)
  labels=np.argmax(final, axis=1)
  return c

In [ ]:
#calculate over training and testing
# since we have not specified an activation function on the last layer
# calling the predict function returns the logits
import timeit
print('Predict on train...')
start1 = timeit.default_timer()
#logits_traind,prob_traind,train_index = calc_attack_matrix(train_data[0:5000],trainX1[0:5000])
prob_trainp= calc_pate(train_data[0:5000],ep[1])
stop1 = timeit.default_timer()
print('time required for members',stop1-start1)
print('Predict on test...')
start2 = timeit.default_timer()
prob_testp= calc_pate(test_data[5000:10000],ep[1])
stop2 = timeit.default_timer()
print('time required for non-members',stop2-start2)


In [ ]:
stop!!

Memuard

In [ ]:
import numpy as np


def memguard(data):
  """ Given confidence vectors, perform memguard post processing to protect from membership inference.
  Note that this defense assumes the strongest defender that can make arbitrary changes to the confidence vector
  so long as it does not change the label. We as well have the (weaker) constrained optimization that will be
  released at a future data.
  Args:
    scores: confidence vectors as 2d numpy array
  Returns: 2d scores protected by memguard.
  """
  print('Predict...')
  scores = model_np.predict(data)
  n_classes = scores.shape[1]
  epsilon = 1e-3
  on_score = (1. / n_classes) + epsilon
  off_score = (1. / n_classes) - (epsilon / (n_classes - 1))
  predicted_labels = np.argmax(scores, axis=-1)
  defended_scores = np.ones_like(scores) * off_score
  defended_scores[np.arange(len(defended_scores)), predicted_labels] = on_score
  return defended_scores

In [ ]:
print('Get Secure Probability Score from MemGuard')

#print('Predict on test...')
#prob_test= model_np.predict()

print("Calculate MemGuard")
prob_trainMG = memguard(train)
prob_testMG = memguard(test)

Model-Stacking

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=2)
skf.get_n_splits(train_data, train_labels)
for train_index, test_index in skf.split(train_data, train_labels):
       train_data1,train_data2=train_data[train_index], train_data[test_index]
       train_labels1,train_labels2=train_labels[train_index], train_labels[test_index]

In [ ]:
#model1
model1=AlexnetModel(shape,num_class)
#model1=make_model()
optimizer = tf.keras.optimizers.SGD(learning_rate)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
#train data for model1
train_y1=to_categorical(train_labels1)

# compile the model
model1.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

history1 = model1.fit(train_data1, train_y1,
                       validation_data=(test_data, test_y),
                       batch_size=128, 
                       epochs=epochs)


In [ ]:
#model2
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
#train data for model2
model2 = RandomForestClassifier(n_estimators=40)
dataF=train_data2.reshape(len(train_data2),32*32*3)
dataS=test_data.reshape(len(test_data),32*32*3)
model2.fit(dataF, train_labels2)
y2=model2.predict(dataS)
accuracy_score(y2,test_labels)

In [ ]:
base_models=[model1,model2]

In [ ]:
def stacked_dataset(d1):
  y1=model1.predict(d1)
  #y1=np.argmax(y1,axis=1)
  d2=d1.reshape(len(d1),32*32*3)
  y2=model2.predict_proba(d2)
  #y=np.hstack((y1,y2))
  y=np.concatenate((y1, y2),axis=1)
  print(y.shape)
  #stackX=np.vstack((d2,y))
  return y

In [ ]:
def fit_stacked_model(members, inputX, inputy):
	# create dataset using ensemble
	stackedX = stacked_dataset(inputX)
	models = LogisticRegression()
	models.fit(stackedX, inputy)
	return models,stackedX

In [ ]:
# fit stacked model using the ensemble
from sklearn.linear_model import LogisticRegression
modelS,stackedX = fit_stacked_model(base_models, test_data[5000:10000], test_labels[5000:10000])

In [ ]:
# evaluate model on test set
yhat =modelS.predict(stackedX) #stacked_prediction(base_models, model, test_labels[5000:10000])
acc = accuracy_score(test_labels[5000:10000], yhat)
print('Stacked Test Accuracy: %.3f' % acc)

In [ ]:
def model_stacking(data,model):
  dataX=stacked_dataset(data)
  prob=model.predict_proba(dataX)
  return prob

In [ ]:
# since we have not specified an activation function on the last layer
# calling the predict function returns the logits
print('Predict on train...')
prob_trainS = model_stacking(train,modelS)
print('Predict on test...')
prob_testS = model_stacking(test,modelS)
print('Compute losses...')
cce = tf.keras.backend.categorical_crossentropy
constant = tf.keras.backend.constant
#yt=trainy_split0[0:nd]
#ys=test_labels[0:nd]
#y_train_onehot = to_categorical(yt)
#y_test_onehot = to_categorical(ys)

In [ ]:
stop!!

MixUp without MMD

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, GlobalAveragePooling2D, \
Dense, Input, Activation, MaxPool2D
from tensorflow.keras import Model
import sklearn.metrics
from numpy.random import default_rng
AUTO = tf.data.AUTOTUNE
rng = default_rng()

In [ ]:
#Convert the data into tensorflow dataset object
# Put aside a few samples to create our validation set
val_samples = 2000
x_val, y_val = test_data[:val_samples], test_y[:val_samples]
#new_x_train, new_y_train = train_data[val_samples:], train_y[val_samples:]

train_ds_one = (
    tf.data.Dataset.from_tensor_slices((train_data, train_y))
    .shuffle(batch_size * 100)
    .batch(batch_size)
)
train_ds_two = (
    tf.data.Dataset.from_tensor_slices(((train_data, train_y)))
    .shuffle(batch_size * 100)
    .batch(batch_size)
)

train_ds = tf.data.Dataset.zip((train_ds_one, train_ds_two))

val_ds = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(batch_size)

test_ds = tf.data.Dataset.from_tensor_slices((test_data, test_y)).batch(batch_size)

In [ ]:
def sample_beta_distribution(size, concentration_0=0.3, concentration_1=0.3):
    gamma_1_sample = tf.random.gamma(shape=[size], alpha=concentration_1)
    gamma_2_sample = tf.random.gamma(shape=[size], alpha=concentration_0)
    return gamma_1_sample / (gamma_1_sample + gamma_2_sample)

In [ ]:
def mix_up(ds_one, ds_two, alpha=0.3):
    # Unpack two datasets
    images_one, labels_one = ds_one
    images_two, labels_two = ds_two
    batch_size = tf.shape(images_one)[0]

    # Sample lambda and reshape it to do the mixup
    l = sample_beta_distribution(batch_size, alpha, alpha)
    x_l = tf.reshape(l, (batch_size, 1, 1, 1))
    y_l = tf.reshape(l, (batch_size, 1))

    # Perform mixup on both images and labels by combining a pair of images/labels
    # (one from each dataset) into one image/label
    images = images_one * x_l + images_two * (1 - x_l)
    labels = labels_one * y_l + labels_two * (1 - y_l)
    return (images, labels)

In [ ]:
train_ds_mu = train_ds.map(
    lambda ds_one, ds_two: mix_up(ds_one, ds_two, alpha=0.3), num_parallel_calls=AUTO
)

In [ ]:
model_mix=AlexnetModel(shape,num_class)
model_mix.summary()

In [ ]:
model_mix.compile(loss="categorical_crossentropy", optimizer="SGD", metrics=["accuracy"])
model_mix.fit(train_ds_mu, validation_data=val_ds, epochs=epochs)

In [ ]:
# since we have not specified an activation function on the last layer
# calling the predict function returns the logits
print('Predict on train...')
prob_trainMi = model_mix.predict(train)
print('Predict on test...')
prob_testMi = model_mix.predict(test)
print('Compute losses...')
cce = tf.keras.backend.categorical_crossentropy
constant = tf.keras.backend.constant

Predict on train...
Predict on test...
Compute losses...


MMD+MixUp

In [ ]:
n=len(train_data)

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, GlobalAveragePooling2D, \
Dense, Input, Activation, MaxPool2D
from tensorflow.keras import Model

import sklearn.metrics

from numpy.random import default_rng
rng = default_rng()
AUTO = tf.data.AUTOTUNE

In [ ]:
val_samples = 2000
x_val, y_val = train_data[30000:40000], train_y[30000:40000]

In [ ]:
val_ds = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(batch_size)

In [ ]:
for images, labels in val_ds:  # only take first element of dataset
    x_val = images.numpy()
    y_val = labels.numpy()

In [ ]:
def sample_beta_distribution(size, concentration_0=0.3, concentration_1=0.3):
    gamma_1_sample = tf.random.gamma(shape=[size], alpha=concentration_1)
    gamma_2_sample = tf.random.gamma(shape=[size], alpha=concentration_0)
    return gamma_1_sample / (gamma_1_sample + gamma_2_sample)

In [ ]:
def mix_up(ds_one, ds_two, alpha=0.3):
    # Unpack two datasets
    images_one, labels_one = ds_one
    images_two, labels_two = ds_two
    batch_size = tf.shape(images_one)[0]

    # Sample lambda and reshape it to do the mixup
    l = sample_beta_distribution(batch_size, alpha, alpha)
    x_l = tf.reshape(l, (batch_size, 1, 1, 1))
    y_l = tf.reshape(l, (batch_size, 1))

    # Perform mixup on both images and labels by combining a pair of images/labels
    # (one from each dataset) into one image/label
    images = images_one * x_l + images_two * (1 - x_l)
    labels = labels_one * y_l + labels_two * (1 - y_l)
    return images, labels

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, images, labels, batch_size=64, shuffle=True):
        super().__init__()
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.key_array = np.arange(self.images.shape[0], dtype=np.uint32)
        self.on_epoch_end()

    def __len__(self):
        return len(self.key_array)//self.batch_size

    def __getitem__(self, index):
        keys = self.key_array[index*self.batch_size:(index+1)*self.batch_size]
        x = np.asarray(self.images[keys], dtype=np.float32)
        y = np.asarray(self.labels[keys], dtype=np.float32)
        return x, y

    def on_epoch_end(self):
        if self.shuffle:
            self.key_array = np.random.permutation(self.key_array)

In [ ]:
generator1 = DataGenerator(images=train_data[0:int(n/2)], labels=train_y[0:int(n/2)], batch_size=batch_size, shuffle=True)
generator2 = DataGenerator(images=train_data[int(n/2):n], labels=train_y[int(n/2):n], batch_size=batch_size, shuffle=True)
n_batches = len(generator1)

In [ ]:
print(n_batches)

195


In [ ]:
!pip install tensorflow-model-remediation

In [ ]:
n_epochs=epochs
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
ce_loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

loss_train = np.zeros(shape=(n_epochs,), dtype=np.float32)
acc_train = np.zeros(shape=(n_epochs,), dtype=np.float32)
loss_val = np.zeros(shape=(n_epochs,))
acc_val = np.zeros(shape=(n_epochs,))

In [ ]:
from tensorflow_model_remediation import min_diff
import tensorflow as tf
mmd_loss=min_diff.losses.MMDLoss()

In [ ]:
def train_ds(new_x_train,new_y_train):
  train_ds_one = (
    tf.data.Dataset.from_tensor_slices((new_x_train, new_y_train))
    .shuffle(batch_size * 100)
    .batch(batch_size)
)
  train_ds_two = (
    tf.data.Dataset.from_tensor_slices((new_x_train, new_y_train))
    .shuffle(batch_size * 100)
    .batch(batch_size)
)
  train_ds = tf.data.Dataset.zip((train_ds_one, train_ds_two))
  return train_ds

In [ ]:
def numpy_img(train_ds_mu):
  for images, labels in train_ds_mu:  # only take first element of dataset
    numpy_images = images.numpy()
    numpy_labels = labels.numpy()
    #l = l.numpy()
    return numpy_images,numpy_labels

In [ ]:
#define the model 
model_mmd=AlexnetModel(shape,num_class)
val_train_dif=30

In [ ]:
for epoch in range(n_epochs):
  epoch_loss_avg = tf.keras.metrics.Mean() # Keeping track of the training loss
  epoch_acc_avg = tf.keras.metrics.Mean() # Keeping track of the training accuracy

  print('==== Epoch #{0:3d} ===='.format(epoch),'/',n_epochs)

  for batch in tqdm(range(n_batches)):
    x, y = generator1[batch]
    x1, y1 = generator2[batch]
    ds=train_ds(x,y)
    ds1=train_ds(x1,y1)
    tds= ds.map(lambda ds_one, ds_two: mix_up(ds_one, ds_two, alpha=0.2), num_parallel_calls=AUTO)
    tds1= ds1.map(lambda ds_one, ds_two: mix_up(ds_one, ds_two, alpha=0.2), num_parallel_calls=AUTO)
    xm,ym=numpy_img(tds)
    #xm1,ym1=numpy_img(tds1)
    #print(x.shape)
    #x,y=mix_upn(x, y, alpha=0.3)


    with tf.GradientTape() as tape: # Forward pass
      y_ = model_mmd(xm, training=True)
      y1_ = model_mmd(x1, training=True)
      yx_ = model_mmd(x, training=True)
      #print(type(y_))
      #print(y)
      loss1 = ce_loss(y_, ym)
      loss2 = mmd_loss(yx_, y1_)
      #print(loss1)
      if val_train_dif>3:
        loss=loss2+loss1
      elif val_train_dif<=3:
        loss=loss1
      #loss = compute_mmd(y1_,y_)/1000
      #losst += loss
    grad = tape.gradient(loss, model_mmd.trainable_variables) # Backpropagation
    optimizer.apply_gradients(zip(grad, model_mmd.trainable_variables)) # Update network weights


    epoch_loss_avg(loss)
    epoch_acc_avg(sklearn.metrics.accuracy_score(y_true=np.argmax(ym, axis=-1), y_pred=np.argmax(y_, axis=-1)))
    
  generator1.on_epoch_end()

  loss_train[epoch] = epoch_loss_avg.result()
  acc_train[epoch] = epoch_acc_avg.result()

  print('---- Training ----')
  print('Loss  =  {0:.3f}'.format(loss_train[epoch]))
  print('Acc   =  {0:.3f}'.format(acc_train[epoch]))

  y_v = model_mmd.predict(x_val) # Validation predictions
  loss_val[epoch] = ce_loss(y_val, y_v).numpy()
  y_true=np.argmax(y_val, axis=-1)
  #print(y_true)
  acc_val[epoch] = sklearn.metrics.accuracy_score(y_true=np.argmax(y_val, axis=-1), y_pred=np.argmax(y_v, axis=-1))
  val_train_dif=(abs(acc_val[epoch]-acc_train[epoch]))*100
  print(val_train_dif)

  print('--- Validation ---')
  print('Loss  =  {0:.3f}'.format(loss_val[epoch]))
  print('Acc   =  {0:.3f}'.format(acc_val[epoch]))

In [ ]:
# since we have not specified an activation function on the last layer
# calling the predict function returns the logits
print('Predict on train...')
prob_trainMi = model_mmd.predict(train)
print('Predict on test...')
prob_testMi = model_mmd.predict(test)
print('Compute losses...')
cce = tf.keras.backend.categorical_crossentropy
constant = tf.keras.backend.constant

Predict on train...
Predict on test...
Compute losses...


Prob-Attack

In [ ]:
prob_trainf=prob_trainMi
prob_testf=prob_testMi
yt=train_labels[0:5000]
ys=test_labels[5000:10000]
attack=3

In [ ]:
(unique, counts) = np.unique(ys, return_counts=True)
frequencies = np.asarray((unique, counts)).T
#frequencies

In [ ]:
# define what variables our attacker should have access to
attack_input = AttackInputData(
  logits_train = prob_trainf,
  logits_test = prob_testf,
  #loss_train = loss_train,
  #loss_test = loss_test,
  labels_train =yt,
  labels_test =ys
)

In [ ]:
# how should the data be sliced
slicing_spec = SlicingSpec(
    entire_dataset = True,
    by_class = False,
    by_percentiles = False,
    by_classification_correctness = True)

In [ ]:
#Give which type of attack you are running
#attack=3 #1=th, 2=LR, 3=MLP

In [ ]:
# define the type of attacker model that we want to use
if attack==1:
  attack_types = [
    AttackType.THRESHOLD_ATTACK
]
elif attack==2:
  attack_types = [
    AttackType.LOGISTIC_REGRESSION
]
elif attack==3:
  attack_types = [
    AttackType.MULTI_LAYERED_PERCEPTRON
]

In [ ]:
# run the attack
attacks_result = mia.run_attacks(attack_input=attack_input,
                                 slicing_spec=slicing_spec,
                                 attack_types=attack_types)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


In [ ]:
# summary by data slice (the best performing attacks per slice are presented)
print(attacks_result.summary(by_slices=True))

In [ ]:
#calculate test accuracy/utility
from sklearn.metrics import accuracy_score
test_labeld_result=np.argmax(prob_testf,axis=1)
accuracy_score(ys,test_labeld_result)

In [ ]:
#calculate train accuracy/utility
from sklearn.metrics import accuracy_score
train_labeld_result=np.argmax(prob_trainf,axis=1)
accuracy_score(yt,train_labeld_result)

In [ ]:
stop!!

GAP Attack

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn import metrics
from sklearn.metrics import accuracy_score

In [ ]:
in1=np.zeros(5000)
in2=np.ones(5000)
Attack_target=np.hstack((in2,in1))
Attack_target

array([1., 1., 1., ..., 0., 0., 0.])

In [ ]:
def Gap_attack(prob_train,prob_test,label_train,label_test):
  result1=np.argmax(prob_train,axis=1)
  result2=np.argmax(prob_test,axis=1)
  result_label=np.hstack((result1,result2))
  target_label=np.hstack((label_train,label_test))
  target_label=np.array(target_label)
  g1=len(target_label)
  target_label=target_label.reshape(g1,1)
  result_label=np.array(result_label)
  result_label=result_label.reshape(g1,1)
  target_label=target_label.astype(int)
  result_label=result_label.astype(int)
  attack_result=np.equal(target_label, result_label)
  attack_result=attack_result.astype(int) 
  fpr, tpr, thresholds = metrics.roc_curve(Attack_target, attack_result, pos_label=1)
  AUC=metrics.auc(fpr, tpr)
  advantage=np.max(abs(fpr-tpr))
  return AUC, advantage

In [ ]:
#gap attack for non-private data
auc,adv=Gap_attack(prob_trainf,prob_testf,yt,ys)
print(auc)
print(adv)

0.6191
0.23819999999999997


In [ ]:
stop!!

Label-Only Attack

In [ ]:
from scipy import ndimage, misc
import matplotlib.pyplot as plt

In [ ]:
#initialize the flags
r=5
target_model=model_np 
private_model=model_mmd #the model that needs to be evaluated
train=train_data[5000:10000]
test=test_data[0:5000]

In [ ]:
def make_fc():
  """Return conv_model as tf.keras.Model, without Softmax layer (only logits).
  To perform
  :param input_shape:
  :param depth:
  :param regularization:
  :param reg_constant:
  :param num_classes:
  :return:
  """
  fc_model = tf.keras.models.Sequential()
  #fc_model.add(tf.keras.layers.Dense(num_classes, kernel_regularizer=k_reg))
  fc_model.add(tf.keras.layers.Dense(10, activation=tf.keras.layers.ReLU(
        negative_slope=1e-2), kernel_initializer='glorot_normal'))
  fc_model.add(keras.layers.Dense(10, activation=tf.keras.layers.ReLU(
        negative_slope=1e-2), kernel_initializer='glorot_normal'))
  fc_model.add(tf.keras.layers.Dense(2, kernel_initializer='glorot_normal'))
  fc_model.add(tf.keras.layers.Softmax())
  return fc_model

In [ ]:
def rotate_image(x1,a):  #angle
  x2=np.zeros(x1.shape)
  x3=np.zeros(x1.shape)
  tp=x1.dtype
  x2=x2.astype(tp)
  x3=x3.astype(tp)
  for i in range(len(x1)):
    out1=ndimage.rotate(x1[i],a,reshape=False)
    out2=ndimage.rotate(x1[i],-a,reshape=False)
    x2[i]=out1
    x3[i]=out2
  return x2,x3

In [ ]:
train_data_rotate1,train_data_rotate2=rotate_image(train_data[5000:10000],r)
test_data_rotate1,test_data_rotate2=rotate_image(test_data[0:5000],r)

In [ ]:
def calc_b(target_model,data,t):
  n=len(data)
  ytr=target_model.predict(data)
  ytr=np.argmax(ytr,axis=1)
  ytr=ytr.reshape(n,1)
  btr=(ytr-t)
  btr=np.where(btr==0,1,0)
  return btr

In [ ]:
t=train_labels[5000:10000].reshape(5000,1)
s=test_labels[0:5000].reshape(5000,1)
T=train_data[5000:10000].reshape(5000,32*32*3)
S=test_data[0:5000].reshape(5000,32*32*3)

In [ ]:
#calculate b
btr1=calc_b(target_model,train_data_rotate1,t)
btr2=calc_b(target_model,train_data_rotate2,t)
btr3=calc_b(target_model,train,t)


bsr1=calc_b(target_model,test_data_rotate1,s)
bsr2=calc_b(target_model,test_data_rotate2,s)
bsr3=calc_b(target_model,test,s)

out1=np.ones(5000)
out2=np.zeros(5000)
label1=np.hstack((out1,out2))
print(label1.shape)
#rearrange b
btr=np.hstack((btr3,btr1,btr2))
#btr=np.transpose(btr)
#print(btr.shape)
bsr=np.hstack((bsr3,bsr1,bsr2))
#bsr=np.transpose(bsr)
b=np.vstack((btr,bsr))
b.shape

(10000,)


(10000, 3)

In [ ]:
total=np.concatenate([t, s])
Bt=np.hstack((b,total))
Bt.shape
label=label1.flatten()
#label=to_categorical(label1)

In [ ]:
from sklearn.model_selection import train_test_split
t_data,s_data,t_label,s_label=train_test_split(Bt, label, test_size=0.1, random_state=42)

In [ ]:
#train a model
modelLB=make_fc()
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# compile the model
modelLB.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
# train the model
history = modelLB.fit(t_data, t_label,
                       validation_data=(s_data, s_label),
                       batch_size=32, 
                       epochs=30)



In [ ]:
from sklearn.metrics import accuracy_score
mem=modelLB.predict(s_data)
mem=np.argmax(mem,axis=1)
#s_label_target=np.argmax(s_label,axis=1)
accuracy_score(mem,s_label)

In [ ]:
#Auc Score
from sklearn.metrics import roc_auc_score
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(mem, s_label, pos_label=1)
metrics.auc(fpr, tpr)

In [ ]:
np.max(abs(fpr-tpr))

In [ ]:
stop!!

In [ ]:
def calc_b(target_model,data,t):    #if only MemGuard
  n=len(data)
  ytr=memguard(data)
  ytr=np.argmax(ytr,axis=1)
  ytr=ytr.reshape(n,1)
  btr=(ytr-t)
  btr=np.where(btr==0,1,0)
  return btr

In [ ]:
def calc_b(private_model,data,t):    #if only model_stacking
  n=len(data)
  ytr=model_stacking(data,private_model)
  ytr=np.argmax(ytr,axis=1)
  ytr=ytr.reshape(n,1)
  btr=(ytr-t)
  btr=np.where(btr==0,1,0)
  return btr

In [ ]:
def calc_b(target_model,data,t):    #if only PATE
  n=len(data)
  ytr=calc_pate(data,ep[0])
  ytr=np.argmax(ytr,axis=1)
  ytr=ytr.reshape(n,1)
  btr=(ytr-t)
  btr=np.where(btr==0,1,0)
  return btr

In [ ]:
#create training and testing rotate data
train_data_rotateT1,train_data_rotateT2=rotate_image(train_data[0:5000],r)
#train_X1_rotateT1,train_X1_rotateT2=rotate_image(x_traind0[0:5000],r)
test_data_rotateT1,test_data_rotateT2=rotate_image(test_data[5000:10000],r)
#test_X1_rotateT1,test_X1_rotateT2=rotate_image(testX1[5000:10000],r)

In [ ]:
t=train_labels[0:5000].reshape(5000,1)
s=test_labels[5000:10000].reshape(5000,1)
T=train_data[0:5000].reshape(5000,32*32*3)
S=test_data[5000:10000].reshape(5000,32*32*3)

In [ ]:
#calculate b
Btr1=calc_b(private_model,train_data_rotateT1,t)
Btr2=calc_b(private_model,train_data_rotateT2,t)
Btr3=calc_b(private_model,train_data[0:5000],t)

Bsr1=calc_b(private_model,test_data_rotateT1,s)
Bsr2=calc_b(private_model,test_data_rotateT2,s)
Bsr3=calc_b(private_model,test_data[5000:10000],s)

out1=np.ones(5000)
out2=np.zeros(5000)
label1=np.hstack((out1,out2))
print(label1.shape)
#rearrange b
Btr=np.hstack((Btr3,Btr1,Btr2))
#btr=np.transpose(btr)
#print(btr.shape)
Bsr=np.hstack((Bsr3,Bsr1,Bsr2))
#bsr=np.transpose(bsr)
B=np.vstack((Btr,Bsr))
B.shape


(10000,)


(10000, 3)

In [ ]:
total=np.concatenate([t, s])
Btotal=np.hstack((B,total))
Btotal.shape
label=label1.flatten()
#label=to_categorical(label1)

In [ ]:
from sklearn.metrics import accuracy_score
mem=modelLB.predict(Btotal)
mem=np.argmax(mem,axis=1)
#s_label_target=np.argmax(s_label,axis=1)
accuracy_score(mem,label)

0.6191

In [ ]:
#Auc Score
from sklearn.metrics import roc_auc_score
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(mem, label, pos_label=1)
metrics.auc(fpr, tpr)

0.6199999468010312

In [ ]:
np.max(abs(fpr-tpr))

0.23999989360206225

In [ ]:
stop!!

Image Translation Attack

In [ ]:
import random
def translate_image(x1,d):  #d=distance
  x2=np.zeros(x1.shape)
  x3=np.zeros(x1.shape)
  x4=np.zeros(x1.shape)
  x5=np.zeros(x1.shape)
  x6=np.zeros(x1.shape)
  tp=x1.dtype
  x2=x2.astype(tp)
  x3=x3.astype(tp)
  x4=x4.astype(tp)
  x5=x5.astype(tp)
  x6=x6.astype(tp)
  for i in range(len(x1)):
    test_x=x1[i]
    i1=1#random.uniform(-d,d)
    j1=d-abs(i1)
    out1=ndimage.interpolation.shift(test_x,np.array([i1,j1,0]))
    #print(i1,j1)
    i2=-1#random.uniform(-d,d)
    j2=d-abs(i2)
    out2=ndimage.interpolation.shift(test_x,np.array([i2,j2,0]))
    i3=0#random.uniform(-d,d)
    j3=d-abs(i3)
    out3=ndimage.interpolation.shift(test_x,np.array([i3,j3,0]))
    i4=0#random.uniform(-d,d)
    j4=-1#d-abs(i4)
    out4=ndimage.interpolation.shift(test_x,np.array([i4,j4,0]))
    i5=0#random.uniform(-d,d)
    j5=0#d-abs(i5)
    out5=ndimage.interpolation.shift(test_x,np.array([i5,j5,0]))
    #_____________________________________________#
    x2[i]=out1
    x3[i]=out2
    x4[i]=out3
    x5[i]=out4
    x6[i]=out5
  return x2,x3,x4,x5,x6

In [ ]:
train_data_trans1,train_data_trans2,train_data_trans3,train_data_trans4,train_data_trans5=translate_image(train_data[5000:10000],1)
test_data_trans1,test_data_trans2,test_data_trans3,test_data_trans4,test_data_trans5=translate_image(test_data[0:5000],1)

In [ ]:
def calc_b(target_model,data,t):
  n=len(data)
  ytr=target_model.predict(data)
  ytr=np.argmax(ytr,axis=1)
  ytr=ytr.reshape(n,1)
  btr=(ytr-t)
  btr=np.where(btr==0,1,0)
  return btr

In [ ]:
t=train_labels[5000:10000].reshape(5000,1)
s=test_labels[0:5000].reshape(5000,1)
T=train_data[5000:10000].reshape(5000,32*32*3)
S=test_data[0:5000].reshape(5000,32*32*3)

In [ ]:
print('Calculate features for training data')
btr1=calc_b(target_model,train_data_trans1,t)
btr2=calc_b(target_model,train_data_trans2,t)
btr3=calc_b(target_model,train_data_trans3,t)
btr4=calc_b(target_model,train_data_trans4,t)
btr5=calc_b(target_model,train_data_trans5,t)
#btr6=calc_b(model_np,train_data[5000:10000],t)

print('Calculate features for testing data')
bsr1=calc_b(target_model,test_data_trans1,s)
bsr2=calc_b(target_model,test_data_trans2,s)
bsr3=calc_b(target_model,test_data_trans3,s)
bsr4=calc_b(target_model,test_data_trans4,s)
bsr5=calc_b(target_model,test_data_trans5,s)
#bsr6=calc_b(model_np,test_data[0:5000],s)

print('rearrange b')
btr=np.hstack((btr5,btr1,btr2,btr3,btr4))
#btr=np.transpose(btr)
#print(btr.shape)
bsr=np.hstack((bsr5,bsr1,bsr2,bsr3,bsr4))
#bsr=np.transpose(bsr)
b=np.vstack((btr,bsr))
b.shape

Calculate features for training data
Calculate features for testing data
rearrange b


(10000, 5)

In [ ]:
total=np.concatenate([t, s])
B=np.hstack((b,total))
B.shape

(10000, 6)

In [ ]:
out1=np.ones(5000)
out2=np.zeros(5000)
label1=np.hstack((out1,out2))
print(label1.shape)
label=label1.flatten()
#label=to_categorical(label1)

(10000,)


In [ ]:
from sklearn.model_selection import train_test_split
t_data,s_data,t_label,s_label=train_test_split(B, label, test_size=0.3, random_state=42)

In [ ]:
#train a model
modelLB=make_fc()
optimizer = tf.keras.optimizers.Adam(learning_rate=.01)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# compile the model
modelLB.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
# train the model
history = modelLB.fit(t_data, t_label,
                       validation_data=(s_data, s_label),
                       batch_size=32, 
                       epochs=30)

In [ ]:
from sklearn.metrics import accuracy_score
mem=modelLB.predict(s_data)
mem=np.argmax(mem,axis=1)
#s_label_target=np.argmax(s_label,axis=1)
accuracy_score(mem,s_label)

In [ ]:
#Auc Score
from sklearn.metrics import roc_auc_score
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(mem, s_label, pos_label=1)
metrics.auc(fpr, tpr)

In [ ]:
np.max(abs(fpr-tpr))

In [ ]:
stop!!!

In [ ]:
def calc_B(target_model,data,t):    #if only MemGuard
  n=len(data)
  ytr=memguard(data)
  ytr=np.argmax(ytr,axis=1)
  ytr=ytr.reshape(n,1)
  btr=(ytr-t)
  btr=np.where(btr==0,1,0)
  return btr

In [ ]:
def calc_B(private_model,data,t):    #if only model_stacking
  n=len(data)
  ytr=model_stacking(data,private_model)
  ytr=np.argmax(ytr,axis=1)
  ytr=ytr.reshape(n,1)
  btr=(ytr-t)
  btr=np.where(btr==0,1,0)
  return btr

In [ ]:
def calc_B(target_model,data,t):    #if only PATE
  n=len(data)
  ytr=calc_pate(data,ep[0])
  ytr=np.argmax(ytr,axis=1)
  ytr=ytr.reshape(n,1)
  btr=(ytr-t)
  btr=np.where(btr==0,1,0)
  return btr

In [ ]:
def calc_B(private_model,data,t):    
  n=len(data)
  ytr=private_model.predict(data)
  ytr=np.argmax(ytr,axis=1)
  ytr=ytr.reshape(n,1)
  btr=(ytr-t)
  btr=np.where(btr==0,1,0)
  return btr

In [ ]:
#Private Model for TA attack
train_data_trans1,train_data_trans2,train_data_trans3,train_data_trans4,train_data_trans5=translate_image(train_data[0:5000],1)
test_data_trans1,test_data_trans2,test_data_trans3,test_data_trans4,test_data_trans5=translate_image(test_data[5000:10000],1)

In [ ]:
t=train_labels[0:5000].reshape(5000,1)
s=test_labels[5000:10000].reshape(5000,1)
T=train_data[0:5000].reshape(5000,32*32*3)
S=test_data[5000:10000].reshape(5000,32*32*3)

In [ ]:
#print('Calculate features for training data')
import timeit
print('Calculate features for training data')
btr1=calc_B(private_model,train_data_trans1,t)
btr2=calc_B(private_model,train_data_trans2,t)
btr3=calc_B(private_model,train_data_trans3,t)
btr4=calc_B(private_model,train_data_trans4,t)
btr5=calc_B(private_model,train_data_trans5,t)
#btr6=calc_b(model_np,train_data[5000:10000],t)

print('Calculate features for testing data')
bsr1=calc_B(private_model,test_data_trans1,s)
bsr2=calc_B(private_model,test_data_trans2,s)
bsr3=calc_B(private_model,test_data_trans3,s)
bsr4=calc_B(private_model,test_data_trans4,s)
bsr5=calc_B(private_model,test_data_trans5,s)
#bsr6=calc_b(model_np,test_data[0:5000],s)

print('rearrange b')
btr=np.hstack((btr5,btr1,btr2,btr3,btr4))
#btr=np.transpose(btr)
#print(btr.shape)
bsr=np.hstack((bsr5,bsr1,bsr2,bsr3,bsr4))
#bsr=np.transpose(bsr)
b1=np.vstack((btr,bsr))
b1.shape

Calculate features for training data
Calculate features for testing data
rearrange b


(10000, 5)

In [ ]:
total=np.concatenate([t, s])
Btotal=np.hstack((b1,total))
Btotal.shape

(10000, 6)

In [ ]:
from sklearn.metrics import accuracy_score
mem=modelLB.predict(Btotal)
mem=np.argmax(mem,axis=1)
#s_label_target=np.argmax(s_label,axis=1)
accuracy_score(mem,label)

In [ ]:
#Auc Score
from sklearn.metrics import roc_auc_score
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(mem, label, pos_label=1)
metrics.auc(fpr, tpr)

In [ ]:
np.max(abs(fpr-tpr))

In [ ]:
stop!!

Boundary Distance Attack

In [ ]:
n1=5000

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_curve

In [ ]:
def continuous_rand_robust(model, dsx,dsy, max_samples=100, noise_samples=250, stddev=0.025, input_dim=shape,
                           num=[1, 2, 3, 4, 5, 10, 20, 30, 40, 50, 75, 100, 150, 250]): #350, 500, 750, 1000, 1500, 2000, 2500]):
  """Calculate robustness to random noise for Adv-x MI attack on continuous-featureed datasets (+ UCI adult).
  :param model: model to approximate distances on (attack).
  :param ds: tf dataset should be either the training set or the test set.
  :param max_samples: maximum number of samples to take from the ds
  :param noise_samples: number of noised samples to take for each sample in the ds.
  :param stddev: the standard deviation to use for Gaussian noise (only for Adult, which has some continuous features)
  :param input_dim: dimension of inputs for the dataset.
  :param num: subnumber of samples to evaluate. max number is noise_samples
  :return: a list of lists. each sublist of the accuracy on up to $num noise_samples.
  """
  robust_accs = [[] for _ in num]
  iter=len(dsx)
  labels=dsy
  for i in range(iter):
    x=dsx[i:i+1]
    noise = stddev * np.random.randn(noise_samples, 32,32,3)
    x_noisy = np.clip(x + noise, 0, 1)
    #print(noise.shape)
    prob=model.predict(x_noisy)
    pred=np.argmax(prob,axis=1)
    #print(pred.shape)
    for idx, n in enumerate(num):
            if n == 0:
              robust_accs[idx].append(1)
            else:
              robust_accs[idx].append(np.mean(pred[:n] == labels[i]))
  return robust_accs

In [ ]:
def continuous_rand_robust_target(model, dsx,dsy, max_samples=100, noise_samples=250, stddev=0.025, input_dim=shape,
                           num=[1, 2, 3, 4, 5, 10, 20, 30, 40, 50, 75, 100, 150, 250]): #350, 500, 750, 1000, 1500, 2000, 2500]):
  """Calculate robustness to random noise for Adv-x MI attack on continuous-featureed datasets (+ UCI adult).
  :param model: model to approximate distances on (attack).
  :param ds: tf dataset should be either the training set or the test set.
  :param max_samples: maximum number of samples to take from the ds
  :param noise_samples: number of noised samples to take for each sample in the ds.
  :param stddev: the standard deviation to use for Gaussian noise (only for Adult, which has some continuous features)
  :param input_dim: dimension of inputs for the dataset.
  :param num: subnumber of samples to evaluate. max number is noise_samples
  :return: a list of lists. each sublist of the accuracy on up to $num noise_samples.
  """
  robust_accs = [[] for _ in num]
  iter=len(dsx)
  labels=dsy
  for i in range(iter):
    x=dsx[i:i+1]
    noise = stddev * np.random.randn(noise_samples, 32,32,3)
    x_noisy = np.clip(x + noise, 0, 1)
    #print(noise.shape)
    #prob=memguard(x_noisy)    #if MemGuard
    #prob=calc_pate(x_noisy,ep[0])  #if PATE
    prob=model.predict(x_noisy)  #others
    #prob=model_stacking(x_noisy,model)  #model stacking
    pred=np.argmax(prob,axis=1)
    #print(pred.shape)
    for idx, n in enumerate(num):
            if n == 0:
              robust_accs[idx].append(1)
            else:
              robust_accs[idx].append(np.mean(pred[:n] == labels[i]))
  return robust_accs

In [ ]:
noise_source_in=continuous_rand_robust(target_model,train_data[0:n1],train_labels[0:n1])
noise_source_out=continuous_rand_robust(target_model,test_data[0:n1],test_labels[0:n1])

In [ ]:
noise_target_in=continuous_rand_robust_target(private_model,train_data[n1:2*n1],train_labels[n1:2*n1])
noise_target_out=continuous_rand_robust_target(private_model,test_data[n1:2*n1],test_labels[n1:2*n1])

In [ ]:
source_m = np.concatenate([np.ones(n1),np.zeros(n1)], axis=0)
target_m = np.concatenate([np.ones(n1),np.zeros(n1)], axis=0)

In [ ]:
def get_max_accuracy(y_true, probs, thresholds=None):
  """Return the max accuracy possible given the correct labels and guesses. Will try all thresholds unless passed.
  Args:
    y_true: True label of `in' or `out' (member or non-member, 1/0)
    probs: The scalar to threshold
    thresholds: In a blackbox setup with a shadow/source model, the threshold obtained by the source model can be passed
      here for attackin the target model. This threshold will then be used.
  Returns: max accuracy possible, accuracy at the threshold passed (if one was passed), the max precision possible,
   and the precision at the threshold passed.
  """
  attack_adv=[]
  if thresholds is None:
    fpr, tpr, thresholds = roc_curve(y_true, probs)
    attack_adv.append(abs(tpr-fpr))
    

  accuracy_scores = []
  precision_scores = []
  for thresh in thresholds:
    accuracy_scores.append(accuracy_score(y_true,
                                          [1 if m > thresh else 0 for m in probs]))
    precision_scores.append(precision_score(y_true, [1 if m > thresh else 0 for m in probs]))

  accuracies = np.array(accuracy_scores)
  precisions = np.array(precision_scores)
  attack_adv=np.array(attack_adv)
  max_accuracy = accuracies.max()
  max_precision = precisions.max()
  #print(type(attack_adv))
  #print(attack_adv)
  if (attack_adv.size !=0):
      max_adv=np.max(attack_adv)
      print(max_adv)
  max_accuracy_threshold = thresholds[accuracies.argmax()]
  max_precision_threshold = thresholds[precisions.argmax()]
  return max_accuracy, max_accuracy_threshold, max_precision, max_precision_threshold

In [ ]:
def get_threshold(source_m, source_stats, target_m, target_stats):
  """ Train a threshold attack model and get teh accuracy on source and target models.
  Args:
    source_m: membership labels for source dataset (1 for member, 0 for non-member)
    source_stats: scalar values to threshold (attack features) for source dataset
    target_m: membership labels for target dataset (1 for member, 0 for non-member)
    target_stats: scalar values to threshold (attack features) for target dataset
  Returns: best acc from source thresh, precision @ same threshold, threshold for best acc,
    precision at the best threshold for precision. all tuned on source model.
  """
  # find best threshold on source data
  acc_source, t, prec_source, tprec = get_max_accuracy(source_m, source_stats)

  # find best accuracy on test data (just to check how much we overfit)
  acc_test, _, prec_test, _ = get_max_accuracy(target_m, target_stats)

  # get the test accuracy at the threshold selected on the source data
  acc_test_t, _, _, _ = get_max_accuracy(target_m, target_stats, thresholds=[t])
  _, _, prec_test_t, _ = get_max_accuracy(target_m, target_stats, thresholds=[tprec])
  print("acc src: {}, acc test (best thresh): {}, acc test (src thresh): {}, thresh: {}".format(acc_source, acc_test,
                                                                                                acc_test_t, t))
  print(
    "prec src: {}, prec test (best thresh): {}, prec test (src thresh): {}, thresh: {}".format(prec_source, prec_test,
                                                                                               prec_test_t, tprec))

  return acc_test_t, prec_test_t, t, tprec

In [ ]:
for i in range(len(noise_source_in)):
            noise_source = np.concatenate([noise_source_in[i], noise_source_out[i]], axis=0)
            noise_target = np.concatenate([noise_target_in[i], noise_target_out[i]], axis=0)
            get_threshold(source_m, noise_source, target_m, noise_target)

In [ ]:
stop!!